In [ ]:
# Find RL_Note path and append sys path
import os, sys
cwd = os.getcwd()
pos = cwd.find('RL_Note')
root_path = cwd[0:pos] + 'RL_Note'
sys.path.append(root_path)

In [1]:
# It is good to refer github following as : 
# https://github.com/shivaverma/OpenAIGym/blob/master/lunar-lander/discrete/lunar_lander.py
import gym
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from pys.utils.memory import ReplayMemory

In [2]:
class DQN(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = Dense(64,activation='relu')
        self.fc2 = Dense(64,activation='relu')
        self.out = Dense(action_size,kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))

    def call(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.out(x)
        return q

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper params for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        
        # Experience Replay
        self.batch_size = 64
        self.train_start = 1000
        self.buffer_size = 10000
        self.memory = ReplayMemory(capacity=self.buffer_size)
        
        self.model        = DQN(self.state_size, self.action_size)
        self.target_model = DQN(self.state_size, self.action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
        self.model.build(input_shape=(None, self.state_size))
        self.model.summary()
        
        self.update_target_model()
        
    def remember(self, state, action, reward, next_state, done):
        state = np.reshape(state, [1, self.state_size])
        next_state = np.reshape(next_state, [1, self.state_size])
        transition = (state, action, reward, next_state, done)
        self.memory.append(transition)
        
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def choose_action(self,state):
        # Exploration and Exploitation
        if (np.random.rand() <= self.epsilon):
            return random.randrange(self.action_size)
        else:
            state = tf.convert_to_tensor([state], dtype=tf.float32)
            return np.argmax(self.model.predict(state))
        
    def train_model(self):
        # Train from Experience Replay
        # Training Condition - Memory Size
        if len(self.memory) < self.train_start:
            return 0.0
        # Decaying Exploration Ratio
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        # Sampling from the memory
        # ER
        mini_batch = random.sample(self.memory, self.batch_size)
        # PER
        # mini_batch, idxs, is_weights = self.memory.sample(self.batch_size)
        
        states      = tf.convert_to_tensor(np.array([sample[0] for sample in mini_batch]))
        actions     = tf.convert_to_tensor(np.array([sample[1] for sample in mini_batch]))
        rewards     = tf.convert_to_tensor(np.array([sample[2] for sample in mini_batch]))
        next_states = tf.convert_to_tensor(np.array([sample[3] for sample in mini_batch]))
        dones       = tf.convert_to_tensor(np.array([sample[4] for sample in mini_batch]))
        
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            predicts = self.model(states)
            one_hot_action = tf.one_hot(actions, self.action_size)
            predicts = tf.reduce_sum(one_hot_action * predicts, axis=1)
            
            target_predicts = self.target_model(next_states)
            target_predicts = tf.stop_gradient(target_predicts)
            
            max_q = np.amax(target_predicts, axis=-1)
            targets = rewards + (1 - dones) * self.discount_factor * max_q
            loss = tf.reduce_mean(tf.square(targets - predicts))
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))
        return loss

    def save_model(self):
        self.model.save_weights('./save_model/LunarLanderv2_dqn_TF', save_format='tf')

In [4]:
%matplotlib tk

ENV_NAME = 'LunarLander-v2'
EPISODES = 1000
END_SCORE = 300

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = DQNAgent(state_size, action_size)
    print('Env Name : ',ENV_NAME)
    print('States {}, Actions {}'
            .format(state_size, action_size))

    scores_avg, scores_raw, episodes, losses = [], [], [], []
    score_avg = 0
    
    end = False
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []

        state = env.reset()

        while not done:
            #env.render()

            # Interact with env.
            action = agent.choose_action(state)
            next_state, reward, done, info = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            loss = agent.train_model()
            state = next_state

            # 
            score += reward
            loss_list.append(loss)
            if done:
                agent.update_target_model()

                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | mem length: {:4d} | epsilon: {:.4f}'
                      .format(e, score_avg, len(agent.memory), agent.epsilon))

                # Save data for plot
                episodes.append(e)
                scores_avg.append(score_avg)
                scores_raw.append(score)
                losses.append(np.mean(loss_list))

                # View data
                plt.subplot(311)
                plt.plot(episodes, scores_avg, 'b')
                plt.xlabel('episode'); plt.ylabel('average score'); plt.grid()
                plt.title('LunarLanderv2 DQN')
                plt.subplot(312)
                plt.plot(episodes, epsilons, 'b')
                plt.xlabel('episode'); plt.ylabel('epsilon'); plt.grid()
                plt.subplot(313)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode'); plt.ylabel('losses') ;plt.grid()
                plt.savefig('./result/LunarLanderv2_dqn_TF.png')

                if score_avg > END_SCORE:
                    agent.save_model()
                    end = True
                    break
        if end == True:
            env.close()
            np.save('./save_model/data/LunarLanderv2_dqn_TF_epi',  episodes)
            np.save('./save_model/data/LunarLanderv2_dqn_TF_score',scores)
            np.save('./save_model/data/LunarLanderv2_dqn_TF_loss', losses)
            print("End")
            break

| mem length: 10000 | epsilon: 0.0100
epi: 131 | score avg -94.06 | mem length: 10000 | epsilon: 0.0100
epi: 132 | score avg -87.22 | mem length: 10000 | epsilon: 0.0100
epi: 133 | score avg -88.26 | mem length: 10000 | epsilon: 0.0100
epi: 134 | score avg -82.47 | mem length: 10000 | epsilon: 0.0100
epi: 135 | score avg -83.44 | mem length: 10000 | epsilon: 0.0100
epi: 136 | score avg -78.64 | mem length: 10000 | epsilon: 0.0100
epi: 137 | score avg -75.25 | mem length: 10000 | epsilon: 0.0100
epi: 138 | score avg -75.88 | mem length: 10000 | epsilon: 0.0100
epi: 139 | score avg -71.96 | mem length: 10000 | epsilon: 0.0100
epi: 140 | score avg -70.25 | mem length: 10000 | epsilon: 0.0100
epi: 141 | score avg -66.72 | mem length: 10000 | epsilon: 0.0100
epi: 142 | score avg -64.76 | mem length: 10000 | epsilon: 0.0100
epi: 143 | score avg -65.78 | mem length: 10000 | epsilon: 0.0100
epi: 144 | score avg -65.12 | mem length: 10000 | epsilon: 0.0100
epi: 145 | score avg -67.05 | mem leng

In [5]:
env.close()